# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Read csv and check data types & structure 

In [5]:
#userid = 10, sessionid = 182
df = pd.read_csv('event_datafile_new.csv', header=0)
df.head()

artist firstName gender  \
0  Barry Tuckwell/Academy of St Martin-in-the-Fie...  Mohammad      M   
1                                       Jimi Hendrix  Mohammad      M   
2                                       Building 429  Mohammad      M   
3                                         The B-52's    Gianna      F   
4                                   Die Mooskirchner    Gianna      F   

   itemInSession   lastName     length level  \
0              0  Rodriguez  277.15873  paid   
1              1  Rodriguez  239.82975  paid   
2              2  Rodriguez  300.61669  paid   
3              0      Jones  321.54077  free   
4              1      Jones  169.29914  free   

                                  location  sessionId  \
0  Sacramento--Roseville--Arden-Arcade, CA        961   
1  Sacramento--Roseville--Arden-Arcade, CA        961   
2  Sacramento--Roseville--Arden-Arcade, CA        961   
3    New York-Newark-Jersey City, NY-NJ-PA        107   
4    New York-Newark-Jersey City, NY-NJ-PA        107   

                                                song  userId  
0  Horn Concerto No. 4 in E flat K495: II. Romanc...      88  
1                            Woodstock Inprovisation      88  
2                               Majesty (LP Version)      88  
3                                         Love Shack      38  
4                                  Frisch und g'sund      38

In [46]:
df1 = df[["sessionId","userId"]]
df1.count() # Count of rows

sessionId    6820
userId       6820
dtype: int64

In [47]:
df1["Sum"] = df["sessionId"].astype(str) + " " + df["userId"].astype(str)
len(df1["Sum"].unique()) # Checking that session_id + user_id is unique

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


776

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [15]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [16]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_keyspace
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)

except Exception as e:
    print(e)

#### Set Keyspace

In [17]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_keyspace')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [36]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
# sessionId = 338, and itemInSession = 4

def insert_into_destination(insert_query, file='event_datafile_new.csv'):
    
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(insert_query, (line[0], line[1], line[2], 
                                           int(line[3]), line[4], float(line[5]), 
                                           line[6], line[7], int(line[8]), 
                                           line[9], int(line[10])))

In [37]:
# TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \

drop_query_1 = """DROP TABLE IF EXISTS music_app_history_1;"""

create_query_1 = """
    CREATE TABLE IF NOT EXISTS music_app_history_1 (
    artist text,
    first_name text,
    gender text,
    item_in_session int,
    last_name text,
    length float,
    level text,
    location text,
    session_id int,
    song text,
    user_id int,
    PRIMARY KEY (session_id, item_in_session)
    );
"""

insert_query_1 = """
    INSERT INTO music_app_history_1 (
    artist,
    first_name,
    gender,
    item_in_session,
    last_name,
    length,
    level,
    location,
    session_id,
    song,
    user_id
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

try:
    session.execute(drop_query_1)
except Exception as e:
    print(e)
    
try:
    session.execute(create_query_1)
except Exception as e:
    print(e)


In [38]:
#call insert statement
insert_into_destination(insert_query_1)

#### Do a SELECT to verify that the data have been inserted into each table

In [43]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query_1 = """
    SELECT
        artist,
        song,
        length
    FROM music_app_history_1
    WHERE session_id = %s
    AND item_in_session = %s;
"""

try:
    rows = session.execute(select_query_1, (338, 4))
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [ ]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
drop_query_2 = """DROP TABLE IF EXISTS music_app_history_2;"""

create_query_2 = """
    CREATE TABLE IF NOT EXISTS music_app_history_2 (
    artist text,
    first_name text,
    gender text,
    item_in_session int,
    last_name text,
    length float,
    level text,
    location text,
    session_id int,
    song text,
    user_id int,
    PRIMARY KEY (user_id, session_id)
    );
"""

insert_query_2 = """
    INSERT INTO music_app_history_2 (
    artist,
    first_name,
    gender,
    item_in_session,
    last_name,
    length,
    level,
    location,
    session_id,
    song,
    user_id
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

# Drop table if exists
try:
    session.execute(drop_query_2)
except Exception as e:
    print(e)
    
    
# create table if not exists
try:
    session.execute(create_query_2)
except Exception as e:
    print(e)
    


In [ ]:
# Call second insert Statement
insert_into_destination(insert_query_2)

In [ ]:
# Select statement to run query
select_query_2 = """
    SELECT
        artist,
        song
    FROM music_app_history_2
    WHERE userid = %s, 
    AND sessionid = %s
    ORDER BY item_in_session, first_name, last_name;
"""

try:
    rows = session.execute(select_query_2, (10, 182))
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song)

In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history 
# who listened to the song 'All Hands Against His Own'
drop_query_3 = """DROP TABLE IF EXISTS music_app_history_3;"""

create_query_3 = """
    CREATE TABLE IF NOT EXISTS music_app_history_3 (
    artist text,
    first_name text,
    gender text,
    item_in_session int,
    last_name text,
    length float,
    level text,
    location text,
    session_id int,
    song text,
    user_id int,
    PRIMARY KEY (session_id, item_in_session)
    );
"""

insert_query_3 = """
    INSERT INTO music_app_history_3 (
    artist,
    first_name,
    gender,
    item_in_session,
    last_name,
    length,
    level,
    location,
    session_id,
    song,
    user_id
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

insert_into_destination(insert_query_3)                   

In [ ]:
select_query_3 = """
    SELECT
        first_name,
        last_name
    FROM music_app_history_3
    WHERE song = %s;
"""

try:
    rows = session.execute(select_query_3, ('All Hands Against His Own'))
except Exception as e:
    print(e)

for row in rows:
    print (row.first_name, row.last_name)

### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions
try:
    session.execute(drop_query_1)
except Exception as e:
    print(e)
    
try:
    session.execute(drop_query_2)
except Exception as e:
    print(e)

try:
    session.execute(drop_query_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()